# Tests

In [1]:
import os
import csv
import copy
import pyhelayers
import utils
print("misc. init ready")

misc. init ready


In [2]:
import os
import csv
import copy
import pyhelayers
import utils
print("misc. init ready")

conf = pyhelayers.HelibConfig()
conf.p = 4079617 # Plaintext prime modulus
# 127 will give 32 slots, 4079617 8192 slots (no, not, but why? - in SEAL-Python this is the reverse-engineered value: BGV works much differently than BFV)
#conf.m = 128 # Cyclotomic polynomial - defines phi(m)
conf.m = 8192 * 2
#conf.m = 1024 * 2
conf.r = 1 # Hensel lifting
conf.L = 1000 # Number of bits of the modulus chain
conf.c = 2 # Number of columns of Key-Switching matrix

print("configuration ready")

he_context = pyhelayers.HelibBgvContext()
he_context.init(conf)
print(he_context)

assert(he_context.get_traits().is_modular_arithmetic)
assert(he_context.get_traits().arithmetic_modulus >= 127)
print ("asserts passed")

misc. init ready
configuration ready
helayers 1.3.0
HELIB BGV context (WITH SECRET KEY)
m=16384 r=1 L=1000 c=2
SecurityLevel=0
Slots=8192

asserts passed


In [3]:
# Create the Encoder using the context.
encoder = pyhelayers.Encoder(he_context)

In [4]:
class ResStorage:
    execTimeTotal = 0
    deviationTotal = 0

import time
import random

num_execs = 500
num_slots = 8192
max_val = 4079617    # 65537 for p = 65537, so it equals p

ccSum = ResStorage()
ccSub = ResStorage()
ccMul = ResStorage()
cSq   = ResStorage()
cNeg  = ResStorage()
cPow  = ResStorage()
cRotR = ResStorage()
cRotL = ResStorage()
cpSum = ResStorage()
cpSub = ResStorage()
cpMul = ResStorage()

def setup():
    global v1, p1, c1, v2, p2, c2
    v1 = [0] * num_slots
    v2 = [0] * num_slots
    for i in range(num_slots):
        v1[i] = random.randint(1,100)
        #v1[i] = 100
    p1 = encoder.encode(v1)
    c1 = encoder.encode_encrypt(v1)

    for i in range(num_slots):
        v2[i] = random.randint(1,100)
        #v2[i] = 100
    p2 = encoder.encode(v2)
    c2 = encoder.encode_encrypt(v2)
    
    
    
# Multiplication
def fun_ccMul():
    setup()
    start = time.time()
    
    c1.multiply(c2)
    c1.multiply(c2)
    
    end = time.time()
    ccMul.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        ccMul.deviationTotal += abs(res[i] - actualRes)
    
# Addition
def fun_ccSum():
    setup()
    start = time.time()
    
    c1.add(c2)
    c1.add(c2)
    
    end = time.time()
    ccSum.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        ccSum.deviationTotal += abs(res[i] - actualRes)
    
# Subtraction
#
# For some reason, in HElayers BGV, negative numbers roll over, and are stored
# only as positive numbers. One way would be to define the top range of the int
# as negativ myself and thus make my own signed integer.
#
# Deduction of the result calculation:
#
# Let's take 30 and 16 as numbers. Subtraction is done two times
# so it is: 30 - 16 - 16
# or:       30 - 2*16
#
# For our case this means:
#
# if: 2*16 > 30 then:
# 
# 30 - 2*16
# 30 - 32
# max_val - (2*16 - 30)
# max_val - (32 - 30)
# max_val - (2)
# 
# Abstractly: v1=30, v2=16
# 
# if: 2*v2 > v1 then:
# 
# max_val - (2*v2 - v1)
#
def fun_ccSub():
    setup()
    start = time.time()
    
    c1.sub(c2)
    c1.sub(c2)
    
    end = time.time()
    ccSub.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    
    for i in range(num_slots):
        actualRes = 0
        if (2 * v2[i] > v1[i]):
            actualRes = max_val - (2 * v2[i] - v1[i])
        else:
            actualRes = v1[i] - v2[i] - v2[i]
        ccSub.deviationTotal += abs(res[i] - actualRes)
    
# Squaring
def fun_cSq():
    setup()
    start = time.time()
    
    c1.square()
    
    end = time.time()
    cSq.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = pow(v1[i],2)
        cSq.deviationTotal += abs(res[i] - actualRes)
    
# Negation
def fun_cNeg():
    setup()
    start = time.time()
    
    c1.negate()
    c1.negate()
    
    end = time.time()
    cNeg.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i]
        cNeg.deviationTotal += abs(res[i] - actualRes)
    
# Left Rotation
def fun_cRotL():
    setup()
    start = time.time()
    
    c1.rotate(1)
    c1.rotate(1)
    
    end = time.time()
    cRotL.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = 0
        if (i + 2 >= num_slots):
            actualRes = v1[i + 2 - num_slots]
        else:
            actualRes = v1[i + 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    
# Right Rotation
def fun_cRotR():
    setup()
    start = time.time()
    
    c1.rotate(-1)
    c1.rotate(-1)
    
    end = time.time()
    cRotR.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = 0
        if (i - 2 < 0):
            actualRes = v1[num_slots + (i - 2)]
        else:
            actualRes = v1[i - 2]
        cRotR.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Addition
def fun_cpSum():
    setup()
    start = time.time()
    
    c1.add_plain(p2)
    c1.add_plain(p2)
    
    end = time.time()
    cpSum.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        cpSum.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Subtraction
#
# Refer to ccSub for explanation of the result calculation.
#
def fun_cpSub():
    setup()
    start = time.time()
    
    c1.sub_plain(p2)
    c1.sub_plain(p2)
    
    end = time.time()
    cpSub.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    
    for i in range(num_slots):
        actualRes = 0
        if (2 * v2[i] > v1[i]):
            actualRes = max_val - (2 * v2[i] - v1[i])
        else:
            actualRes = v1[i] - v2[i] - v2[i]
        ccSub.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Multiplication
def fun_cpMul():
    setup()
    start = time.time()
    
    c1.multiply_plain(p2)
    c1.multiply_plain(p2)
    
    end = time.time()
    cpMul.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        cpMul.deviationTotal += abs(res[i] - actualRes)
    
#if __name__ == "__main__":
    
for i in range(num_execs):
    fun_ccSum()
print("ccSum completed")
for i in range(num_execs):
    fun_ccSub()
print("ccSub completed")
for i in range(num_execs):
    fun_ccMul()
print("ccMul completed")
for i in range(num_execs):
    fun_cSq()
print("cSq completed")
for i in range(num_execs):
    fun_cNeg()
print("cNeg completed")
for i in range(num_execs):
    fun_cRotL()
print("cRotL completed")
for i in range(num_execs):
    fun_cRotR()
print("cRotR completed")
for i in range(num_execs):
    fun_cpSum()
print("cpSum completed")
for i in range(num_execs):
    fun_cpSub()
print("cpSub completed")
for i in range(num_execs):
    fun_cpMul()
print("cpMul completed")

ccSum completed
ccSub completed
ccMul completed
cSq completed
cNeg completed
cRotL completed
cRotR completed
cpSum completed
cpSub completed
cpMul completed


In [5]:
    print('Execution time in milliseconds:')
    print()
    print('Addition:')    # Formula: Total time or deviation for all cycles / operations per cycle
    print('Average execution time: {:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
    print()
    print('Subtraction:')
    print('Average execution time: {:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
    print()
    print('Multiplication:')
    print('Average execution time: {:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
    print()
    print('Squaring:')    # Square only has 1 operation per cycle, otherwise the configuration needs to be changed
    print('Average execution time: {:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
    print()
    print('Negation:')
    print('Average execution time: {:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
    print()
    print('Left Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
    print()
    print('Right Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Addition:')
    print('Average execution time: {:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Subtraction:')
    print('Average execution time: {:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Multiplication:')
    print('Average execution time: {:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))
    
    PRINT_RAW_NUMBERS = True
    if (PRINT_RAW_NUMBERS):
        print()
        print()
        print('Raw numbers:')
        print()
        print('Average execution time:')
        print('{:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
        print('{:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print()
        print('Average deviation:')
        print('{:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
        print('{:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))

Execution time in milliseconds:

Addition:
Average execution time: 0.574405
Average deviation:      0.000000

Subtraction:
Average execution time: 0.590532
Average deviation:      0.000000

Multiplication:
Average execution time: 143.159352
Average deviation:      0.000000

Squaring:
Average execution time: 111.080257
Average deviation:      0.000000

Negation:
Average execution time: 0.462940
Average deviation:      0.000000

Left Rotation:
Average execution time: 225.597397
Average deviation:      0.000000

Right Rotation:
Average execution time: 149.627825
Average deviation:      0.000000

Ciphertext-Plain Addition:
Average execution time: 27.632771
Average deviation:      0.000000

Ciphertext-Plain Subtraction:
Average execution time: 38.966958
Average deviation:      0.000000

Ciphertext-Plain Multiplication:
Average execution time: 24.003798
Average deviation:      0.000000


Raw numbers:

Average execution time:
0.574405
0.590532
143.159352
111.080257
0.462940
225.597397
149.627

In [6]:
def setup2():
    global v1, p1, c1, v2, p2, c2
    v1 = [0] * num_slots
    v2 = [0] * num_slots
    for i in range(num_slots):
        #v1[i] = random.randint(1,100)
        v1[i] = 100
    p1 = encoder.encode(v1)
    c1 = encoder.encode_encrypt(v1)

    for i in range(num_slots):
        #v2[i] = random.randint(1,100)
        v2[i] = 100
    p2 = encoder.encode(v2)
    c2 = encoder.encode_encrypt(v2)
    
    
    
# Multiplication
def fun_ccMul2():
    setup2()
    start = time.time()
    
    c1.multiply(c2)
    c1.multiply(c2)
    
    end = time.time()
    ccMul.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        ccMul.deviationTotal += abs(res[i] - actualRes)
    
# Addition
def fun_ccSum2():
    setup2()
    start = time.time()
    
    c1.add(c2)
    c1.add(c2)
    
    end = time.time()
    ccSum.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        ccSum.deviationTotal += abs(res[i] - actualRes)
    
# Subtraction
#
# For some reason, in HElayers BGV, negative numbers roll over, and are stored
# only as positive numbers. One way would be to define the top range of the int
# as negativ myself and thus make my own signed integer.
#
# Deduction of the result calculation:
#
# Let's take 30 and 16 as numbers. Subtraction is done two times
# so it is: 30 - 16 - 16
# or:       30 - 2*16
#
# For our case this means:
#
# if: 2*16 > 30 then:
# 
# 30 - 2*16
# 30 - 32
# max_val - (2*16 - 30)
# max_val - (32 - 30)
# max_val - (2)
# 
# Abstractly: v1=30, v2=16
# 
# if: 2*v2 > v1 then:
# 
# max_val - (2*v2 - v1)
#
def fun_ccSub2():
    setup2()
    start = time.time()
    
    c1.sub(c2)
    c1.sub(c2)
    
    end = time.time()
    ccSub.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    
    for i in range(num_slots):
        actualRes = 0
        if (2 * v2[i] > v1[i]):
            actualRes = max_val - (2 * v2[i] - v1[i])
        else:
            actualRes = v1[i] - v2[i] - v2[i]
        ccSub.deviationTotal += abs(res[i] - actualRes)
    
# Squaring
def fun_cSq2():
    setup2()
    start = time.time()
    
    c1.square()
    
    end = time.time()
    cSq.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = pow(v1[i],2)
        cSq.deviationTotal += abs(res[i] - actualRes)
    
# Negation
def fun_cNeg2():
    setup2()
    start = time.time()
    
    c1.negate()
    c1.negate()
    
    end = time.time()
    cNeg.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i]
        cNeg.deviationTotal += abs(res[i] - actualRes)
    
# Left Rotation
def fun_cRotL2():
    setup2()
    start = time.time()
    
    c1.rotate(1)
    c1.rotate(1)
    
    end = time.time()
    cRotL.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = 0
        if (i + 2 >= num_slots):
            actualRes = v1[i + 2 - num_slots]
        else:
            actualRes = v1[i + 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    
# Right Rotation
def fun_cRotR2():
    setup2()
    start = time.time()
    
    c1.rotate(-1)
    c1.rotate(-1)
    
    end = time.time()
    cRotR.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = 0
        if (i - 2 < 0):
            actualRes = v1[num_slots + (i - 2)]
        else:
            actualRes = v1[i - 2]
        cRotR.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Addition
def fun_cpSum2():
    setup2()
    start = time.time()
    
    c1.add_plain(p2)
    c1.add_plain(p2)
    
    end = time.time()
    cpSum.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        cpSum.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Subtraction
#
# Refer to ccSub for explanation of the result calculation.
#
def fun_cpSub2():
    setup2()
    start = time.time()
    
    c1.sub_plain(p2)
    c1.sub_plain(p2)
    
    end = time.time()
    cpSub.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    
    for i in range(num_slots):
        actualRes = 0
        if (2 * v2[i] > v1[i]):
            actualRes = max_val - (2 * v2[i] - v1[i])
        else:
            actualRes = v1[i] - v2[i] - v2[i]
        ccSub.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Multiplication
def fun_cpMul2():
    setup2()
    start = time.time()
    
    c1.multiply_plain(p2)
    c1.multiply_plain(p2)
    
    end = time.time()
    cpMul.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        cpMul.deviationTotal += abs(res[i] - actualRes)
    
#if __name__ == "__main__":
    
for i in range(num_execs):
    fun_ccSum2()
print("ccSum completed")
for i in range(num_execs):
    fun_ccSub2()
print("ccSub completed")
for i in range(num_execs):
    fun_ccMul2()
print("ccMul completed")
for i in range(num_execs):
    fun_cSq2()
print("cSq completed")
for i in range(num_execs):
    fun_cNeg2()
print("cNeg completed")
for i in range(num_execs):
    fun_cRotL2()
print("cRotL completed")
for i in range(num_execs):
    fun_cRotR2()
print("cRotR completed")
for i in range(num_execs):
    fun_cpSum2()
print("cpSum completed")
for i in range(num_execs):
    fun_cpSub2()
print("cpSub completed")
for i in range(num_execs):
    fun_cpMul2()
print("cpMul completed")

ccSum completed
ccSub completed
ccMul completed
cSq completed
cNeg completed
cRotL completed
cRotR completed
cpSum completed
cpSub completed
cpMul completed


In [7]:
    print('Execution time in milliseconds:')
    print()
    print('Addition:')    # Formula: Total time or deviation for all cycles / operations per cycle
    print('Average execution time: {:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
    print()
    print('Subtraction:')
    print('Average execution time: {:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
    print()
    print('Multiplication:')
    print('Average execution time: {:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
    print()
    print('Squaring:')    # Square only has 1 operation per cycle, otherwise the configuration needs to be changed
    print('Average execution time: {:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
    print()
    print('Negation:')
    print('Average execution time: {:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
    print()
    print('Left Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
    print()
    print('Right Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Addition:')
    print('Average execution time: {:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Subtraction:')
    print('Average execution time: {:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Multiplication:')
    print('Average execution time: {:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))
    
    PRINT_RAW_NUMBERS = True
    if (PRINT_RAW_NUMBERS):
        print()
        print()
        print('Raw numbers:')
        print()
        print('Average execution time:')
        print('{:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
        print('{:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print()
        print('Average deviation:')
        print('{:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
        print('{:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))

Execution time in milliseconds:

Addition:
Average execution time: 1.141045
Average deviation:      0.000000

Subtraction:
Average execution time: 1.146679
Average deviation:      0.000000

Multiplication:
Average execution time: 258.167802
Average deviation:      0.000000

Squaring:
Average execution time: 196.036979
Average deviation:      0.000000

Negation:
Average execution time: 0.931942
Average deviation:      0.000000

Left Rotation:
Average execution time: 436.420230
Average deviation:      0.000000

Right Rotation:
Average execution time: 285.941758
Average deviation:      0.000000

Ciphertext-Plain Addition:
Average execution time: 48.082252
Average deviation:      0.000000

Ciphertext-Plain Subtraction:
Average execution time: 71.116051
Average deviation:      0.000000

Ciphertext-Plain Multiplication:
Average execution time: 44.642202
Average deviation:      0.000000


Raw numbers:

Average execution time:
1.141045
1.146679
258.167802
196.036979
0.931942
436.420230
285.941